## Imports

In [1]:
# packages
import mne
import scipy.io
from scipy.signal import hilbert
import numpy as np

# local
import os

## Functions

In [2]:
def get_egi_file_paths(dir_path):
    file_list = []

    for dir in os.listdir(dir_path):
        if os.path.isdir(os.path.join(dir_path, dir)):
            for file in os.listdir(os.path.join(dir_path, dir)):
                if ".RAW" in file and not file.startswith("."):
                    file_list.append(os.path.join(dir_path, dir, file))

    return file_list

In [ ]:
def preprocess_egi(raw):
    raw.load_data()
    #raw._data = mne.filter.detrend(raw._data, axis=1, order=1)
    raw.apply_function(lambda x: mne.filter.detrend(x, axis=0, order=1), picks='all')

    # bandpass filter
    raw.filter(l_freq=0.25, h_freq=40, picks='all')

    return raw

## Variables

In [4]:
# main directory for where all .RAW files can be found
raw_egi_files = r"D:\MOTORWP4_DATASET\EGI"
output_path = r"D:\dilon_data\edf"
# channel used to create the new .edf file
channels = [
    "E214", "E41",    # frontal
    "E183", "E59",    # central
    "E149", "E124",   # occipital
    "E10", "E54",     # eog
    "E240", "E243",   # emg
    "E190", "E94"     # references
]

channel_name = {
    "E214": "F1", "E41": "F2",        # frontal
    "E183": "C1", "E59": "C2",        # central
    "E149": "O1", "E124": "O2",       # occipital
    "E10": "EOG1", "E54": "EOG2",     # eog
    "E240": "EMG1", "E243": "EMG2",   # emg
}

## Main

In [5]:
# get all .RAW filepaths
print("Retrieving list of all egi .RAW files in directory.")
egi_file_list = get_egi_file_paths(raw_egi_files)
print("Files retrieved.")
print(len(egi_file_list))

Retrieving list of all egi .RAW files in directory.
Files retrieved.
217


In [6]:
# iterating through all files
for egi_file in egi_file_list:
    print(egi_file)
    subject = egi_file.split("\\")[-1].split(" ")[0]
    if os.path.exists(os.path.join(output_path, f"{subject}.edf")):
        continue
    else:
        print(f"Creating .edf file for subject: {subject}")

        # load .RAW egi file
        raw_egi = mne.io.read_raw_egi(
            egi_file, 
            preload=False
        )

        # get channel selection
        raw_egi = raw_egi.pick(channels)
        # pre-process the data
        raw_egi = preprocess_egi(raw_egi)

        # # get integer indices
        # indices = mne.pick_channels(raw_egi.ch_names, ["E190", "E94"])

        # # take average of reference channels and create a new ref channel
        # reference = mne.channels.combine_channels(
        #     raw_egi, 
        #     groups={"ref": indices},
        #     method="mean"
        # )

        # raw_egi.drop_channels(["E190", "E94"])

        # # create bipolar channels with reference 
        # for key, value in channel_name.items():
        #     raw_egi.add_channels([reference], force_update_info=True)
        #     mne.set_bipolar_reference(
        #         raw_egi, 
        #         anode=key, 
        #         cathode="ref", 
        #         ch_name=value,
        #         #drop_refs=True,
        #         copy=False
        #     )

        raw_egi.set_channel_types(
            {
                "E214": "eeg", "E41": "eeg",    # frontal
                "E183": "eeg", "E59": "eeg",    # central
                "E149": "eeg", "E124": "eeg",   # occipital
                "E10": "eog", "E54": "eog",     # eog
                "E240": "emg", "E243": "emg",   # emg
                "E190": "misc", "E94": "misc"   # reference
            }
        )

        raw_egi.export(os.path.join(output_path, f"{subject}.edf"), fmt="edf")


D:\MOTORWP4_DATASET\EGI\S40\ELECTR_S40_2 20210412 2254.RAW
D:\MOTORWP4_DATASET\EGI\S40\MOTORWP4_S40_2 20210412 23.RAW
D:\MOTORWP4_DATASET\EGI\S35\MOTORWP4_S35_1 20201211 2224.RAW
D:\MOTORWP4_DATASET\EGI\S35\MOTORWP4_S35_2 20201214 22.RAW
D:\MOTORWP4_DATASET\EGI\S41\ELECTR_S41_1 20210416 2206.RAW
D:\MOTORWP4_DATASET\EGI\S41\ELECTR_S41_2 20210419 2157.RAW
D:\MOTORWP4_DATASET\EGI\S41\MOTORWP4_S41_1 20210416 22.RAW
D:\MOTORWP4_DATASET\EGI\S41\MOTORWP4_S41_2 20210419 22.RAW
D:\MOTORWP4_DATASET\EGI\S90\ELECTR_S90_1 20220628 2232.RAW
D:\MOTORWP4_DATASET\EGI\S90\ELECTR_S90_2 20220701 2204.RAW
D:\MOTORWP4_DATASET\EGI\S90\MOTORWP4_S90_2 20220701 23.RAW
D:\MOTORWP4_DATASET\EGI\S42\ELECTR_S42_1 20210423 2202.RAW
D:\MOTORWP4_DATASET\EGI\S42\ELECTR_S42_2 20210426 2124.RAW
D:\MOTORWP4_DATASET\EGI\S42\MOTORWP4_S42_1 20210423 22.RAW
D:\MOTORWP4_DATASET\EGI\S42\MOTORWP4_S42_2 20210426 22.RAW
D:\MOTORWP4_DATASET\EGI\S36\MOTORWP4_S36_1 20210125 22.RAW
D:\MOTORWP4_DATASET\EGI\S36\MOTORWP4_S36_2 20210128 22

C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:45: RuntimeWarning: The unit for channel(s) E190, E94 has changed from V to NA.
  raw_egi.set_channel_types(
C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:56: RuntimeWarning: EDF format requires equal-length data blocks, so 0.684 seconds of edge values were appended to all channels when writing the final block.
  raw_egi.export(os.path.join(output_path, f"{subject}.edf"), fmt="edf")


D:\MOTORWP4_DATASET\EGI\S85\ELECTR_S85_1 20220725 2150.RAW
Creating .edf file for subject: ELECTR_S85_1
Reading EGI header from D:\MOTORWP4_DATASET\EGI\S85\ELECTR_S85_1 20220725 2150.RAW...
    Reading events ...
    Assembling measurement info ...
    Excluding events {} ...
Reading 0 ... 42764  =      0.000 ...   171.056 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.25 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.25
- Lower transition bandwidth: 0.25 Hz (-6 dB cutoff frequency: 0.12 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (13.204 s)

D:\MOTORWP4_DATASET\EGI\S85\ELECTR_S85_2 20220728 2151.RAW
Creating .edf file for subject: ELECTR_S85

C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:45: RuntimeWarning: The unit for channel(s) E190, E94 has changed from V to NA.
  raw_egi.set_channel_types(
C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:56: RuntimeWarning: EDF format requires equal-length data blocks, so 0.94 seconds of edge values were appended to all channels when writing the final block.
  raw_egi.export(os.path.join(output_path, f"{subject}.edf"), fmt="edf")


    Assembling measurement info ...
    Excluding events {} ...
Reading 0 ... 43614  =      0.000 ...   174.456 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.25 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.25
- Lower transition bandwidth: 0.25 Hz (-6 dB cutoff frequency: 0.12 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (13.204 s)

D:\MOTORWP4_DATASET\EGI\S85\MOTORWP4_S85_1 20220725 22.RAW
Creating .edf file for subject: MOTORWP4_S85_1
Reading EGI header from D:\MOTORWP4_DATASET\EGI\S85\MOTORWP4_S85_1 20220725 22.RAW...
    Reading events ...


...

C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:45: RuntimeWarning: The unit for channel(s) E190, E94 has changed from V to NA.
  raw_egi.set_channel_types(
C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:56: RuntimeWarning: EDF format requires equal-length data blocks, so 0.204 seconds of edge values were appended to all channels when writing the final block.
  raw_egi.export(os.path.join(output_path, f"{subject}.edf"), fmt="edf")


D:\MOTORWP4_DATASET\EGI\S63\MOTORWP4_S63_1 20210212 23.RAW
Creating .edf file for subject: MOTORWP4_S63_1
Reading EGI header from D:\MOTORWP4_DATASET\EGI\S63\MOTORWP4_S63_1 20210212 23.RAW...
    Reading events ...
    Assembling measurement info ...
    Excluding events {} ...
Reading 0 ... 7642622  =      0.000 ... 30570.488 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.25 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.25
- Lower transition bandwidth: 0.25 Hz (-6 dB cutoff frequency: 0.12 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (13.204 s)



C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:45: RuntimeWarning: The unit for channel(s) E190, E94 has changed from V to NA.
  raw_egi.set_channel_types(
C:\Users\andri\AppData\Local\Temp\ipykernel_14436\313904278.py:56: RuntimeWarning: EDF format requires equal-length data blocks, so 0.508 seconds of edge values were appended to all channels when writing the final block.
  raw_egi.export(os.path.join(output_path, f"{subject}.edf"), fmt="edf")


D:\MOTORWP4_DATASET\EGI\S63\MOTORWP4_S63_2 20210215 23.RAW
Creating .edf file for subject: MOTORWP4_S63_2
Reading EGI header from D:\MOTORWP4_DATASET\EGI\S63\MOTORWP4_S63_2 20210215 23.RAW...
    Reading events ...
    Assembling measurement info ...
    Excluding events {} ...
Reading 0 ... 7013710  =      0.000 ... 28054.840 secs...


RuntimeError: Incorrect number of samples (23225335 != 24999936), please report this error to MNE-Python developers